In [1]:
from pytessel import PyTessel
import numpy as np
import trimesh
import pythreejs as p3

def f(x,y,z,X0,Y0,Z0):
    """
    Single metaball function
    """
    return 1 / ((x - X0)**2 + (y - Y0)**2 + (z - Z0)**2)

def icosahedron_field(x,y,z):
    """
    Produce a scalar field for the icosahedral metaballs
    """
    phi = (1 + np.sqrt(5)) / 2
    vertices = [
        [0,1,phi],
        [0,-1,-phi],
        [0,1,-phi],
        [0,-1,phi],
        [1,phi,0],
        [-1,-phi,0],
        [1,-phi,0],
        [-1,phi,0],
        [phi,0,1],
        [-phi,0,-1],
        [phi,0,-1],
        [-phi,0,1]
    ]

    xx,yy,zz = np.meshgrid(x,y,z)
    field = np.zeros_like(xx)
    for v in vertices:
        field += f(xx,yy,zz,v[0], v[1],v[2])

    return xx,yy,zz,field

pytessel = PyTessel()

# generate scalar field and tesselate it
x = np.linspace(-3, 3, 101)
xx, yy, zz, field = icosahedron_field(x,x,x)
unitcell = np.diag(np.ones(3) * 3 * 2)
pytessel = PyTessel()
isovalue = 3.75
vertices, normals, indices = pytessel.marching_cubes(field.flatten(), field.shape, unitcell.flatten(), isovalue)
pytessel.write_ply('test.ply', vertices, normals, indices)

# Load a 3D mesh from a file (Change to your actual file)
mesh = trimesh.load_mesh("test.ply")  # Change to "model.stl" if using STL

# Extract vertices and faces
vertices = mesh.vertices
faces = mesh.faces

# Convert to Three.js format
geometry = p3.BufferGeometry(
    attributes={
        "position": p3.BufferAttribute(vertices.astype(np.float32), normalized=False),
        "index": p3.BufferAttribute(faces.astype(np.uint32).ravel(), normalized=False),
        #"color": p3.BufferAttribute(np.repeat(colors, 3, axis=0).astype(np.float32), normalized=False)
    }
)

# Solid Mesh with Normal-Based Colors
solid_material = p3.MeshStandardMaterial(
    color="lightblue",
    side="DoubleSide"
)

mesh_object = p3.Mesh(geometry=geometry, material=solid_material)

# Wireframe Overlay
wireframe_material = p3.LineBasicMaterial(color="black", linewidth=1.0)  # Thin black lines
wireframe = p3.LineSegments(
    p3.EdgesGeometry(geometry),  # Extracts edges from geometry
    wireframe_material
)

# Lighting setup
ambient_light = p3.AmbientLight(color="white", intensity=0.8)  # Soft background light

# Create Scene
scene = p3.Scene(children=[mesh_object, wireframe, ambient_light, p3.AxesHelper(size=2)])
camera = p3.PerspectiveCamera(position=[5, 5, 5], fov=50)
controller = p3.OrbitControls(controlling=camera)

# Render in Jupyter Notebook with Better Performance
renderer = p3.Renderer(
    scene=scene, camera=camera, controls=[controller], 
    width=700, height=700, antialias=True
)

display(renderer)

C:\Users\ifilot\AppData\Local\anaconda3\Lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
C:\Users\ifilot\AppData\Local\anaconda3\Lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(position=(5.0, 5.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.…